## Prerequisites - Environment Setup

Before starting this workshop, you need to set up a Python virtual environment. Follow these steps:

### Step 1: Create Virtual Environment
From the **root folder** of this repository, run:
```bash
python -m venv .venv
```

### Step 2: Activate Virtual Environment
- **Windows**: `.venv\Scripts\activate`
- **macOS/Linux**: `source .venv/bin/activate`

### Step 3: Upgrade pip
```bash
pip install --upgrade pip
```

### Step 4: Install Jupyter Kernel Support
```bash
pip install ipykernel
```

### Step 5: Select Virtual Environment in VS Code
1. Click on the **kernel selector** in the top-right corner of this notebook
2. Select **"Select Another Kernel..."**
3. Choose **"Python Environments..."**
4. Select the `.venv` environment you just created

### Step 6: Verify Setup
Once you've selected the correct kernel, you can proceed with the workshop. The first code cell will install the required dependencies.

---

# LangChain Agents Workshop: From Simple to Advanced

Welcome to this comprehensive workshop where you'll learn to build sophisticated AI agents using **LangChain** and **Azure OpenAI**!

## What You'll Build
1. **Generic Agent** - Start with the fundamentals of conversational AI
2. **Azure AI Foundry Agent** - Level up with cloud-powered AI capabilities  
3. **Group Chat System** - Master advanced multi-agent orchestration

## Quick Setup
1. **Run the cells below** to install dependencies and configure your environment
2. **Provide your Azure OpenAI credentials** when prompted  
3. **Follow along step-by-step** to build increasingly sophisticated agents

**Ready?** Let's build some amazing AI agents together!

---

**Note**: This workshop uses Azure OpenAI. Make sure you have access to Azure OpenAI services and the required credentials.

In [6]:
# Step 1: Install Dependencies
# Set up the environment for LangChain agents

import os, sys, subprocess, pathlib

nb_dir = pathlib.Path().resolve()
project_root = nb_dir.parents[2] if (len(nb_dir.parents) >= 2) else nb_dir
lc_dir = nb_dir  # this notebook lives in Backend/python/langchain
shared_dir = lc_dir.parent / "shared"
req_file = lc_dir / "requirements.txt"

print(f"Notebook dir: {nb_dir}")
print(f"Project root: {project_root}")
print(f"Using requirements: {req_file}")
print(f"Shared package dir: {shared_dir}")

def run_command(cmd):
    print(f"\nRunning: {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        raise SystemExit(f"Command failed with exit code {result.returncode}")
    if result.stdout:
        print(f"Output: {result.stdout}")

# Install dependencies using pip magic commands (keeps kernel environment clean)
try:
    import IPython
    get_ipython  # Verify we're in IPython/Jupyter
    
    print("\nInstalling requirements...")
    if req_file.exists():
        get_ipython().run_line_magic("pip", f"install -r {req_file}")
        print("✓ Requirements installed!")
    else:
        print("! requirements.txt not found; skipping dependency install.")
    
    print("\nInstalling shared library...")
    if (shared_dir / "setup.py").exists():
        get_ipython().run_line_magic("pip", f"install -e {shared_dir}")
        print("✓ Shared library installed!")
    else:
        print("! Shared library setup.py not found; skipping -e install.")
        
except Exception as e:
    print(f"! IPython magic not available, falling back to subprocess: {e}")
    if req_file.exists():
        run_command(f"python -m pip install -r \"{req_file}\"")
    if (shared_dir / "setup.py").exists():
        run_command(f"python -m pip install -e \"{shared_dir}\"")

print("\n✓ Dependencies installation complete!")

Notebook dir: C:\repo\nhcloud\agentcon-workshop\Backend\python\langchain
Project root: C:\repo\nhcloud\agentcon-workshop
Using requirements: C:\repo\nhcloud\agentcon-workshop\Backend\python\langchain\requirements.txt
Shared package dir: C:\repo\nhcloud\agentcon-workshop\Backend\python\shared

Installing requirements...
Note: you may need to restart the kernel to use updated packages.
✓ Requirements installed!

Installing shared library...
Note: you may need to restart the kernel to use updated packages.
✓ Requirements installed!

Installing shared library...
Obtaining file:///C:/repo/nhcloud/agentcon-workshop/Backend/python/shared
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'don

In [7]:
# Step 2: Load Azure OpenAI Configuration
# Simple check for required environment variables

import os
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
load_dotenv()

# Required Azure OpenAI environment variables
required_vars = {
    "AZURE_OPENAI_ENDPOINT": "Your Azure OpenAI endpoint URL",
    "AZURE_OPENAI_API_KEY": "Your Azure OpenAI API key", 
    "AZURE_OPENAI_DEPLOYMENT_NAME": "Your model deployment name (e.g., gpt-4o-mini)"
}

# Check if all required variables are present
missing_vars = []
for var_name, description in required_vars.items():
    value = os.getenv(var_name)
    if value:
        print(f"✓ {var_name}: Found")
    else:
        print(f"✗ {var_name}: Missing")
        missing_vars.append(var_name)

if missing_vars:
    print(f"\n! Missing required environment variables: {', '.join(missing_vars)}")
    print("\nTo fix this, create a .env file in this directory with:")
    print("AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/")
    print("AZURE_OPENAI_API_KEY=your_api_key_here")
    print("AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini")
    print("\nThen restart this notebook kernel and run this cell again.")
    raise ValueError("Required Azure OpenAI configuration missing!")

# Show successful configuration
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print(f"\n✓ Azure OpenAI configuration loaded successfully!")
print(f"Endpoint: {endpoint}")
print(f"Deployment: {deployment}")
print("Ready to create your Generic Agent!")

✓ AZURE_OPENAI_ENDPOINT: Found
✓ AZURE_OPENAI_API_KEY: Found
✓ AZURE_OPENAI_DEPLOYMENT_NAME: Found

✓ Azure OpenAI configuration loaded successfully!
Endpoint: https://aoai-devdemo.openai.azure.com/
Deployment: gpt-4.1
Ready to create your Generic Agent!


In [8]:
# Step 3: Verify Project Structure
# Ensure all required components are in place

import sys
from pathlib import Path

def check_structure():
    """Verify the project has the expected structure."""
    print("Verifying project structure...")
    
    # Add shared modules to path
    shared_path = str(nb_dir.parent / "shared")
    if shared_path not in sys.path:
        sys.path.insert(0, shared_path)
        print(f"✓ Added shared path: {shared_path}")
    
    # Check key directories and files
    checks = [
        (shared_dir, "Shared library directory"),
        (lc_dir / "agents", "LangChain agents directory"),
        (lc_dir / "config.yml", "Configuration file"),
        (req_file, "Requirements file")
    ]
    
    all_good = True
    for path, description in checks:
        if path.exists():
            print(f"✓ {description}: {path}")
        else:
            print(f"! {description}: {path} (missing)")
            if description == "Shared library directory":
                print("  This is expected - the shared path exists at the parent level")
            else:
                all_good = False
    
    return all_good

# Verify structure
structure_ok = check_structure()

if structure_ok:
    print("\n✓ Project structure verified!")
    print("Ready to build amazing agents!")
else:
    print("\n! Some components are missing but we can continue")
    print("The core workshop will still work!")

Verifying project structure...
✓ Shared library directory: C:\repo\nhcloud\agentcon-workshop\Backend\python\shared
✓ LangChain agents directory: C:\repo\nhcloud\agentcon-workshop\Backend\python\langchain\agents
✓ Configuration file: C:\repo\nhcloud\agentcon-workshop\Backend\python\langchain\config.yml
✓ Requirements file: C:\repo\nhcloud\agentcon-workshop\Backend\python\langchain\requirements.txt

✓ Project structure verified!
Ready to build amazing agents!


In [9]:
# Step 4: Quick Smoke Test
# Test Azure OpenAI connectivity

print("Running quick smoke test with Azure OpenAI...")

try:
    # Test imports
    print("Testing LangChain imports...")
    from langchain_openai import AzureChatOpenAI
    from langchain.schema import HumanMessage
    print("✓ LangChain imports successful")
    
    # Test Azure OpenAI connection
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    
    if endpoint and api_key and deployment:
        print("Testing Azure OpenAI connection...")
        
        # Create a simple test LLM
        llm = AzureChatOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version="2024-02-01",
            deployment_name=deployment,
            temperature=0.1
        )
        
        # Simple test message
        test_message = HumanMessage(content="Hello! Please respond with 'Azure OpenAI is working!'")
        response = llm.invoke([test_message])
        
        print("✓ Azure OpenAI connection successful!")
        print(f"Test message sent")
        print(f"Response: {response.content}")
        
    else:
        print("! Azure OpenAI not fully configured")
        print("Please ensure AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, and AZURE_OPENAI_DEPLOYMENT_NAME are set")
    
    # Test shared library (optional)
    try:
        from shared import AgentRegistry, AgentConfig, AgentMessage
        print("✓ Shared library available")
    except ImportError:
        print("i Shared library not available (optional)")
    
    print("\n✓ Smoke test completed!")
    print("You're ready to build your first Generic Agent!")
    
except Exception as e:
    print(f"✗ Smoke test failed: {e}")
    print("Please check your Azure OpenAI configuration and try again.")
    print("\nCommon issues:")
    print("- Check your endpoint URL format")
    print("- Verify your API key is correct")
    print("- Ensure your deployment name matches your Azure OpenAI resource")

Running quick smoke test with Azure OpenAI...
Testing LangChain imports...
✓ LangChain imports successful
Testing Azure OpenAI connection...
✓ LangChain imports successful
Testing Azure OpenAI connection...
✓ Azure OpenAI connection successful!
Test message sent
Response: Azure OpenAI is working!
i Shared library not available (optional)

✓ Smoke test completed!
You're ready to build your first Generic Agent!
✓ Azure OpenAI connection successful!
Test message sent
Response: Azure OpenAI is working!
i Shared library not available (optional)

✓ Smoke test completed!
You're ready to build your first Generic Agent!


# Workshop Part 1: Building Your First Generic Agent

Welcome to your first agent! We'll start with a **Generic Agent** - a simple but powerful conversational AI that can handle various topics and tasks.

## What You'll Learn
- Basic agent architecture with LangChain
- How to create prompts that guide agent behavior
- Message handling and conversation flow
- Memory management for conversation context
- Testing and interacting with your agent

## Why Start with a Generic Agent?
A "Generic Agent" is perfect for beginners because it:
- **Versatile**: Can handle many different types of conversations
- **Simple**: Easy to understand and modify
- **Foundation**: Provides core concepts for more specialized agents
- **Memory**: Maintains conversation history for context

## 🏗️ Step 1.1: Import Required Libraries

First, let's import all the libraries we need for our genetic agent.

## 🧠 Step 1.2: Create the Genetic Agent

Now let's create our genetic agent with evolutionary capabilities!

In [10]:
# Generic Agent Implementation
# A simple but powerful conversational agent using Azure OpenAI

import asyncio
from datetime import datetime
from typing import List, Dict, Any
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

class GenericAgent:
    """
    A simple generic conversational agent that can handle various topics.
    Perfect for beginners to understand LangChain basics.
    """
    
    def __init__(self, name: str = "GenericBot"):
        self.name = name
        self.conversation_history = []
        
        # Initialize the LLM with Azure OpenAI (easier for beginners)
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2024-02-01",
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
            temperature=0.7
        )
        
        # Create a simple prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self._get_system_prompt()),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}")
        ])
        
        # Initialize memory to remember conversation
        self.memory = ConversationBufferMemory(
            memory_key="history",
            return_messages=True,
            input_key="input"
        )
        
        print(f"{self.name} initialized and ready to chat!")
        print("This agent can help with various topics and remembers our conversation.")
    
    def _get_system_prompt(self) -> str:
        """Get the system prompt that defines the agent's behavior."""
        return f"""You are {self.name}, a helpful and friendly AI assistant.

Your characteristics:
- Helpful and informative
- Conversational and engaging  
- Clear and concise in responses
- Patient and understanding
- Use emojis occasionally to be friendly

Guidelines:
- Always be polite and respectful
- Provide accurate information
- If you're unsure about something, say so
- Keep responses helpful but not too long
- Remember the conversation context

You're here to assist users with various questions and tasks!"""
    
    async def process_message(self, user_input: str) -> str:
        """Process a user message and generate a helpful response."""
        try:
            # Create the conversation chain
            chain = self.prompt | self.llm
            
            # Get the conversation history from memory
            history = self.memory.chat_memory.messages
            
            # Generate response using the chain
            response = await chain.ainvoke({
                "input": user_input,
                "history": history
            })
            
            # Save the conversation to memory
            self.memory.save_context(
                {"input": user_input},
                {"output": response.content}
            )
            
            return response.content
            
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation so far."""
        message_count = len(self.memory.chat_memory.messages)
        return f"""Conversation Summary for {self.name}:
        
Total messages: {message_count}
Agent status: Active and ready
Memory: Conversation history preserved

This agent demonstrates basic LangChain concepts:
- LLM integration (Azure OpenAI)
- Memory management
- Prompt templates
- Conversation chains
"""

# Create and test the generic agent
print("Creating your first Generic Agent...")
generic_agent = GenericAgent("Assistant")
print("✓ Generic Agent created successfully!")
print("Ready to start chatting! The agent will remember your conversation.")

Creating your first Generic Agent...
Assistant initialized and ready to chat!
This agent can help with various topics and remembers our conversation.
✓ Generic Agent created successfully!
Ready to start chatting! The agent will remember your conversation.
Assistant initialized and ready to chat!
This agent can help with various topics and remembers our conversation.
✓ Generic Agent created successfully!
Ready to start chatting! The agent will remember your conversation.


C:\Users\udai\AppData\Local\Temp\3\ipykernel_14596\221718903.py:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


In [13]:
# Step 1.3: Test Your Generic Agent
# Let's have a conversation with your agent!

async def chat_with_generic_agent():
    """Interactive chat function with the generic agent."""
    print("Starting conversation with your Generic Agent")
    print("Type 'quit' to end the conversation")
    print("Type 'summary' to see conversation statistics")
    print("-" * 50)
    
    while True:
        try:
            # Get user input
            user_message = input("\nYou: ").strip()
            
            if user_message.lower() == 'quit':
                print("Thanks for chatting! Goodbye!")
                break
            elif user_message.lower() == 'summary':
                print(generic_agent.get_conversation_summary())
                continue
            elif not user_message:
                print("Please enter a message or 'quit' to exit.")
                continue
            
            # Get agent response
            print("Assistant: ", end="", flush=True)
            response = await generic_agent.process_message(user_message)
            print(response)
            
        except KeyboardInterrupt:
            print("\nChat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")

# Example: Run a quick demo conversation
async def demo_conversation():
    """Run a demonstration conversation."""
    print("Running a quick demo conversation...")
    
    demo_messages = [
        "Hello! Can you introduce yourself?",
        "What can you help me with?",
        "Tell me a fun fact about AI"
    ]
    
    for message in demo_messages:
        print(f"\nDemo User: {message}")
        response = await generic_agent.process_message(message)
        print(f"Assistant: {response}")
    
    print(f"\n{generic_agent.get_conversation_summary()}")

print("Choose an option:")
print("1. Run demo conversation: await demo_conversation()")
print("2. Start interactive chat: await chat_with_generic_agent()")

Choose an option:
1. Run demo conversation: await demo_conversation()
2. Start interactive chat: await chat_with_generic_agent()


In [14]:
# Step 1.4: Interactive Generic Agent Testing
# Try your own questions with the generic agent

def interactive_generic_test():
    """Interactive testing function for the generic agent."""
    print("Try these example interactions with your Generic Agent:")
    print()
    
    examples = [
        "General Question: 'What's the weather like today?' (it will explain it needs external data)",
        "Creative Request: 'Help me brainstorm ideas for a birthday party'", 
        "Technical Help: 'Explain what an API is in simple terms'",
        "Thoughtful Discussion: 'What are the benefits of learning new languages?'",
        "Educational: 'Can you teach me about renewable energy?'"
    ]
    
    for example in examples:
        print(f"  • {example}")
    
    print()
    print("Notice how the agent:")
    print("  - Remembers your conversation")
    print("  - Adapts its responses to be helpful")
    print("  - Provides clear, friendly answers")
    print("  - Maintains context throughout the chat")
    
    return examples

# Show the examples
test_examples = interactive_generic_test()

print("\nReady to test? Use either:")
print("• await demo_conversation()  # For automated demo") 
print("• await chat_with_generic_agent()  # For interactive chat")

Try these example interactions with your Generic Agent:

  • General Question: 'What's the weather like today?' (it will explain it needs external data)
  • Creative Request: 'Help me brainstorm ideas for a birthday party'
  • Technical Help: 'Explain what an API is in simple terms'
  • Thoughtful Discussion: 'What are the benefits of learning new languages?'
  • Educational: 'Can you teach me about renewable energy?'

Notice how the agent:
  - Remembers your conversation
  - Adapts its responses to be helpful
  - Provides clear, friendly answers
  - Maintains context throughout the chat

Ready to test? Use either:
• await demo_conversation()  # For automated demo
• await chat_with_generic_agent()  # For interactive chat


# Workshop Part 2: Azure AI Foundry Agent

Now let's level up! You'll build a sophisticated **Azure AI Foundry Agent** that leverages cloud-powered AI capabilities.

## What You'll Learn
- Advanced Azure AI integration patterns
- Multi-modal AI capabilities (text, reasoning, analysis)
- Professional agent architecture
- Real-world deployment considerations

## The Power of Azure AI Foundry
Azure AI Foundry provides:
- **Scale**: Handle thousands of conversations
- **Intelligence**: Advanced reasoning capabilities  
- **Security**: Enterprise-grade protection
- **Tools**: Rich ecosystem of AI services

Let's build a production-ready agent!

In [15]:
# Azure AI Foundry Agent Implementation
# A sophisticated agent using Azure AI's full capabilities

from dataclasses import dataclass
from enum import Enum
from typing import Optional, List, Dict
import json
import time

# Configuration management
@dataclass
class FoundryConfig:
    """Configuration for Azure AI Foundry integration."""
    endpoint: str
    api_key: str
    deployment_name: str
    api_version: str = "2024-06-01"
    temperature: float = 0.7
    max_tokens: int = 1500
    timeout: int = 30

# Agent capabilities enumeration
class AgentCapability(Enum):
    """Available capabilities for the Azure AI Foundry Agent."""
    CONVERSATIONAL = "conversational"
    ANALYTICAL = "analytical"
    CREATIVE = "creative"
    TECHNICAL = "technical"
    RESEARCH = "research"

# Message types for structured communication
@dataclass
class AgentMessage:
    """Structured message format for agent communication."""
    content: str
    message_type: str = "user"
    timestamp: Optional[str] = None
    metadata: Optional[Dict] = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = str(int(time.time()))

# Context management for conversation state
@dataclass 
class ConversationContext:
    """Manages conversation state and context."""
    messages: List[AgentMessage]
    session_id: str
    capabilities_used: List[AgentCapability]
    performance_metrics: Dict[str, any]
    
    def __post_init__(self):
        if not self.messages:
            self.messages = []
        if not self.capabilities_used:
            self.capabilities_used = []
        if not self.performance_metrics:
            self.performance_metrics = {
                "total_messages": 0,
                "avg_response_time": 0.0,
                "success_rate": 1.0
            }

class AzureAIFoundryAgent:
    """
    Advanced Azure AI Foundry Agent with enterprise capabilities.
    
    This agent demonstrates:
    - Professional error handling
    - Performance monitoring
    - Structured conversations
    - Multi-capability routing
    - Production-ready patterns
    """
    
    def __init__(self, config: FoundryConfig, session_id: str = None):
        """Initialize the Azure AI Foundry Agent."""
        self.config = config
        self.session_id = session_id or f"session_{int(time.time())}"
        self.context = ConversationContext(
            messages=[],
            session_id=self.session_id,
            capabilities_used=[],
            performance_metrics={}
        )
        
        # Initialize Azure OpenAI client
        from langchain_openai import AzureChatOpenAI
        self.llm = AzureChatOpenAI(
            azure_endpoint=config.endpoint,
            api_key=config.api_key,
            azure_deployment=config.deployment_name,
            api_version=config.api_version,
            temperature=config.temperature,
            max_tokens=config.max_tokens,
            timeout=config.timeout
        )
        
        print(f"Azure AI Foundry Agent initialized successfully!")
        print(f"Session ID: {self.session_id}")
        print(f"Endpoint: {config.endpoint}")
        print(f"Deployment: {config.deployment_name}")
    
    def detect_capability_needed(self, message: str) -> AgentCapability:
        """Analyze message to determine which capability is needed."""
        message_lower = message.lower()
        
        # Analysis keywords
        analysis_keywords = ["analyze", "compare", "evaluate", "assess", "review", "examine"]
        creative_keywords = ["create", "brainstorm", "design", "imagine", "generate", "write"]
        technical_keywords = ["code", "programming", "technical", "algorithm", "debug", "implement"]
        research_keywords = ["research", "find", "search", "investigate", "study", "explore"]
        
        if any(keyword in message_lower for keyword in analysis_keywords):
            return AgentCapability.ANALYTICAL
        elif any(keyword in message_lower for keyword in creative_keywords):
            return AgentCapability.CREATIVE
        elif any(keyword in message_lower for keyword in technical_keywords):
            return AgentCapability.TECHNICAL
        elif any(keyword in message_lower for keyword in research_keywords):
            return AgentCapability.RESEARCH
        else:
            return AgentCapability.CONVERSATIONAL
    
    def create_capability_prompt(self, capability: AgentCapability, message: str) -> str:
        """Create specialized prompts based on the detected capability."""
        base_context = f"""You are an Azure AI Foundry Agent with advanced capabilities.
Session ID: {self.session_id}
Current capability mode: {capability.value}

User message: {message}
"""
        
        capability_instructions = {
            AgentCapability.ANALYTICAL: """
Focus on: Deep analysis, structured thinking, data interpretation, and insights.
Provide: Clear reasoning, evidence-based conclusions, and actionable recommendations.
""",
            AgentCapability.CREATIVE: """
Focus on: Innovation, brainstorming, creative solutions, and imaginative thinking.
Provide: Original ideas, creative approaches, and inspirational content.
""",
            AgentCapability.TECHNICAL: """
Focus on: Technical accuracy, implementation details, best practices, and solutions.
Provide: Code examples, technical explanations, and practical guidance.
""",
            AgentCapability.RESEARCH: """
Focus on: Information gathering, fact-checking, comprehensive overviews, and sources.
Provide: Well-researched answers, multiple perspectives, and reliable information.
""",
            AgentCapability.CONVERSATIONAL: """
Focus on: Natural conversation, helpful responses, and user engagement.
Provide: Friendly, informative, and contextually appropriate responses.
"""
        }
        
        return base_context + capability_instructions[capability]
    
    async def process_message(self, user_message: str) -> str:
        """Process a user message with full capability detection and routing."""
        start_time = time.time()
        
        try:
            # Create user message object
            user_msg = AgentMessage(content=user_message, message_type="user")
            self.context.messages.append(user_msg)
            
            # Detect needed capability
            capability = self.detect_capability_needed(user_message)
            if capability not in self.context.capabilities_used:
                self.context.capabilities_used.append(capability)
            
            # Create specialized prompt
            specialized_prompt = self.create_capability_prompt(capability, user_message)
            
            # Get AI response
            response = await self.llm.ainvoke([{"role": "user", "content": specialized_prompt}])
            response_content = response.content
            
            # Create assistant message object
            assistant_msg = AgentMessage(
                content=response_content, 
                message_type="assistant",
                metadata={"capability_used": capability.value}
            )
            self.context.messages.append(assistant_msg)
            
            # Update performance metrics
            response_time = time.time() - start_time
            self.context.performance_metrics["total_messages"] += 1
            current_avg = self.context.performance_metrics.get("avg_response_time", 0)
            total_msgs = self.context.performance_metrics["total_messages"]
            new_avg = (current_avg * (total_msgs - 1) + response_time) / total_msgs
            self.context.performance_metrics["avg_response_time"] = new_avg
            
            return response_content
            
        except Exception as e:
            # Update error metrics
            self.context.performance_metrics["success_rate"] = (
                (self.context.performance_metrics.get("total_messages", 0) - 1) /
                max(self.context.performance_metrics.get("total_messages", 1), 1)
            )
            
            error_response = f"I encountered an error processing your request: {str(e)}"
            error_msg = AgentMessage(
                content=error_response,
                message_type="error",
                metadata={"error_type": type(e).__name__}
            )
            self.context.messages.append(error_msg)
            
            return error_response
    
    def get_session_analytics(self) -> Dict:
        """Get comprehensive session analytics and insights."""
        return {
            "session_id": self.session_id,
            "total_messages": len(self.context.messages),
            "capabilities_used": [cap.value for cap in self.context.capabilities_used],
            "performance_metrics": self.context.performance_metrics,
            "conversation_length": len([msg for msg in self.context.messages if msg.message_type == "user"]),
            "success_rate": self.context.performance_metrics.get("success_rate", 1.0),
            "avg_response_time": f"{self.context.performance_metrics.get('avg_response_time', 0):.2f}s"
        }
    
    def export_conversation(self) -> Dict:
        """Export the complete conversation for analysis or storage."""
        return {
            "session_info": {
                "session_id": self.session_id,
                "created_at": self.context.messages[0].timestamp if self.context.messages else None,
                "config": {
                    "endpoint": self.config.endpoint,
                    "deployment": self.config.deployment_name,
                    "temperature": self.config.temperature
                }
            },
            "messages": [
                {
                    "content": msg.content,
                    "type": msg.message_type,
                    "timestamp": msg.timestamp,
                    "metadata": msg.metadata
                }
                for msg in self.context.messages
            ],
            "analytics": self.get_session_analytics()
        }

print("Azure AI Foundry Agent class defined successfully!")
print("Key features:")
print("- Capability-based routing (analytical, creative, technical, research, conversational)")
print("- Performance monitoring and analytics")
print("- Structured conversation management")
print("- Enterprise-ready error handling")
print("- Session export capabilities")

Azure AI Foundry Agent class defined successfully!
Key features:
- Capability-based routing (analytical, creative, technical, research, conversational)
- Performance monitoring and analytics
- Structured conversation management
- Enterprise-ready error handling
- Session export capabilities


## Step 2.2: Test Azure AI Foundry Capabilities

Let's test the different capabilities of our Azure AI Foundry agent!

In [16]:
# Test Different Azure AI Capabilities
# Demonstrate the agent's intelligent capability routing

async def test_foundry_capabilities():
    """Test different capabilities of the Azure AI Foundry agent."""
    
    test_scenarios = [
        {
            "category": "Analytical",
            "prompt": "Analyze the pros and cons of cloud computing vs on-premise solutions",
            "expected_capability": AgentCapability.ANALYTICAL
        },
        {
            "category": "Creative",
            "prompt": "Design an innovative app concept for sustainable living",
            "expected_capability": AgentCapability.CREATIVE
        },
        {
            "category": "Technical",
            "prompt": "My Python script is running slowly. How can I troubleshoot and fix it?",
            "expected_capability": AgentCapability.TECHNICAL
        },
        {
            "category": "Research",
            "prompt": "What is quantum computing and how does it work?",
            "expected_capability": AgentCapability.RESEARCH
        },
        {
            "category": "Conversational",
            "prompt": "Hello! How are you doing today?",
            "expected_capability": AgentCapability.CONVERSATIONAL
        }
    ]
    
    print("Testing Azure AI Foundry Agent Capabilities")
    print("=" * 60)
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n{i}. {scenario['category']} Test")
        print(f"Prompt: {scenario['prompt']}")
        print("-" * 40)
        
        # Process the message
        response = await foundry_agent.process_message(scenario['prompt'])
        
        # Display response (first 200 characters)
        print(f"Response: {response[:200]}...")
        print("-" * 60)
    
    # Show final stats
    print(f"\nAgent Performance Summary:")
    stats = foundry_agent.get_session_analytics()
    for key, value in stats.items():
        print(f"   {key}: {value}")

print("Ready to test capabilities!")
print("Run: await test_foundry_capabilities()")

Ready to test capabilities!
Run: await test_foundry_capabilities()


In [17]:
# Step 2.3: Advanced Foundry Agent Testing
# Test specific capabilities and interactive chat

async def chat_with_foundry_agent():
    """Interactive chat function with the Azure AI Foundry agent."""
    print("Starting conversation with your Azure AI Foundry Agent")
    print("This agent automatically detects the best capability for your request!")
    print("Type 'quit' to end, 'analytics' to see session stats")
    print("-" * 60)
    
    while True:
        try:
            # Get user input
            user_message = input("\nYou: ").strip()
            
            if user_message.lower() == 'quit':
                print("Thanks for testing the Azure AI Foundry Agent! Goodbye!")
                break
            elif user_message.lower() == 'analytics':
                stats = foundry_agent.get_session_analytics()
                print("\nSession Analytics:")
                for key, value in stats.items():
                    print(f"  {key}: {value}")
                continue
            elif not user_message:
                print("Please enter a message or 'quit' to exit.")
                continue
            
            # Get agent response
            print("Foundry Agent: ", end="", flush=True)
            response = await foundry_agent.process_message(user_message)
            print(response)
            
        except KeyboardInterrupt:
            print("\nChat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"Error: {e}")

print("Ready for advanced testing!")
print("Run: await chat_with_foundry_agent()")

Ready for advanced testing!
Run: await chat_with_foundry_agent()


In [18]:
# Interactive Azure AI Foundry Testing
# Try your own prompts with capability detection

def interactive_foundry_examples():
    """Show examples for interactive testing with the Foundry agent."""
    
    print("Try These Examples with the Azure AI Foundry Agent:")
    print()
    
    examples = {
        "Analytical Examples": [
            "Compare Python vs JavaScript for web development",
            "Evaluate the security implications of cloud storage",
            "Analyze the benefits of microservices architecture"
        ],
        "Creative Examples": [
            "Design a futuristic smart city concept",
            "Create an innovative solution for remote work collaboration",
            "Imagine a new type of user interface"
        ],
        "Technical Examples": [
            "My website loads slowly, how can I optimize it?",
            "How to handle database connection errors in production?",
            "Best practices for debugging complex applications"
        ],
        "Research Examples": [
            "What are the latest trends in artificial intelligence?",
            "How does blockchain technology work?",
            "Explain quantum computing in simple terms"
        ]
    }
    
    for category, prompts in examples.items():
        print(f"{category}:")
        for prompt in prompts:
            print(f"  • {prompt}")
        print()
    
    print("Notice how the agent will:")
    print("- Automatically detect the best capability for each request")
    print("- Provide specialized responses based on the capability")
    print("- Track analytics and performance metrics")
    print("- Maintain conversation context and history")
    
    return examples

# Show examples
examples = interactive_foundry_examples()

print("\nReady to test? Use:")
print("• await test_foundry_capabilities()  # Run automated tests")
print("• await chat_with_foundry_agent()    # Interactive conversation")

Try These Examples with the Azure AI Foundry Agent:

Analytical Examples:
  • Compare Python vs JavaScript for web development
  • Evaluate the security implications of cloud storage
  • Analyze the benefits of microservices architecture

Creative Examples:
  • Design a futuristic smart city concept
  • Create an innovative solution for remote work collaboration
  • Imagine a new type of user interface

Technical Examples:
  • My website loads slowly, how can I optimize it?
  • How to handle database connection errors in production?
  • Best practices for debugging complex applications

Research Examples:
  • What are the latest trends in artificial intelligence?
  • How does blockchain technology work?
  • Explain quantum computing in simple terms

Notice how the agent will:
- Automatically detect the best capability for each request
- Provide specialized responses based on the capability
- Track analytics and performance metrics
- Maintain conversation context and history

Ready to t

In [19]:
# Bridge: Combining Generic and Foundry Agents
# See how different agent types can work together

async def agent_collaboration_demo():
    """Demonstrate how generic and foundry agents can collaborate."""
    
    print("Agent Collaboration Demonstration")
    print("=" * 50)
    
    collaboration_prompt = "How can AI agents work together effectively?"
    
    print(f"Collaboration Question: {collaboration_prompt}")
    print()
    
    # Get response from generic agent
    print("Generic Agent responds:")
    generic_response = await generic_agent.process_message(collaboration_prompt)
    print(f"   {generic_response[:200]}...")
    print()
    
    # Get response from foundry agent
    print("Foundry Agent analyzes:")
    foundry_prompt = f"Analyze this perspective on AI collaboration: {generic_response[:100]}... and provide additional insights on {collaboration_prompt}"
    foundry_response = await foundry_agent.process_message(foundry_prompt)
    print(f"   {foundry_response[:200]}...")
    
    print("\nKey Insight: Different agents bring different strengths!")
    print("Generic Agent: Conversational, accessible responses")
    print("Foundry Agent: Structured, capability-driven responses")
    print("Together: More comprehensive and nuanced conversations!")

print("Ready to test collaboration!")
print("Run: await agent_collaboration_demo()")

Ready to test collaboration!
Run: await agent_collaboration_demo()


# Workshop Part 3: Advanced Group Chat System

Welcome to the final challenge! You'll build a sophisticated **Multi-Agent Group Chat System** where different AI personalities collaborate to solve complex problems.

## What You'll Master
- Multi-agent orchestration and coordination
- Dynamic conversation flow management
- Agent specialization and role assignment
- Consensus building and conflict resolution
- Real-time collaboration patterns

## The Power of Agent Teams
Group chat systems enable:
- **Specialization**: Each agent brings unique expertise
- **Collaboration**: Agents build on each other's ideas
- **Efficiency**: Parallel processing of complex problems
- **Diversity**: Different perspectives and approaches

Let's build the future of AI collaboration!

In [21]:
# Advanced Group Chat System Implementation
# Multi-agent collaboration with orchestration and coordination

import uuid
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Dict, Optional, Set, Any
from enum import Enum

class AgentRole(Enum):
    """Different roles agents can play in group chat."""
    FACILITATOR = "facilitator"
    EXPERT = "expert"
    CRITIC = "critic"
    SYNTHESIZER = "synthesizer"
    CREATIVE = "creative"

@dataclass
class ChatMessage:
    """Enhanced message structure for group chat."""
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    sender: str = ""
    content: str = ""
    role: AgentRole = AgentRole.EXPERT
    timestamp: datetime = field(default_factory=datetime.now)
    references: List[str] = field(default_factory=list)  # IDs of messages this responds to
    confidence: float = 0.0
    metadata: Dict[str, Any] = field(default_factory=dict)

class GroupChatOrchestrator:
    """Orchestrates conversation flow between multiple agents."""
    
    def __init__(self):
        self.agents = {}
        self.conversation_history = []
        self.active_topics = set()
        self.conversation_id = str(uuid.uuid4())
        
    def add_agent(self, agent, role: AgentRole):
        """Add an agent to the group chat with a specific role."""
        agent_id = f"{agent.name}_{role.value}"
        self.agents[agent_id] = {
            "agent": agent,
            "role": role,
            "message_count": 0,
            "last_active": None
        }
        print(f"Added {agent.name} as {role.value}")
        
    def get_conversation_context(self, last_n_messages: int = 5) -> str:
        """Get recent conversation context for agents."""
        recent_messages = self.conversation_history[-last_n_messages:]
        context = "Recent conversation:\n"
        for msg in recent_messages:
            context += f"{msg.sender} ({msg.role.value}): {msg.content}\n"
        return context
        
    async def facilitate_discussion(self, topic: str, max_rounds: int = 3) -> List[ChatMessage]:
        """Facilitate a structured discussion on a topic."""
        print(f"Starting group discussion on: {topic}")
        print("=" * 60)
        
        # Initialize the discussion
        discussion_messages = []
        self.active_topics.add(topic)
        
        # Round 1: Each agent provides initial perspective
        print("Round 1: Initial Perspectives")
        for agent_id, agent_info in self.agents.items():
            agent = agent_info["agent"]
            role = agent_info["role"]
            
            # Create role-specific prompt
            prompt = self._create_role_prompt(topic, role, self.get_conversation_context())
            
            # Get agent response
            if hasattr(agent, 'process_message'):
                response_content = await agent.process_message(prompt)
                if hasattr(response_content, 'content'):
                    response_content = response_content.content
            else:
                response_content = f"[{agent.name} would respond here based on their {role.value} role]"
            
            # Create chat message
            message = ChatMessage(
                sender=agent.name,
                content=response_content,
                role=role,
                confidence=0.8,
                metadata={"round": 1, "topic": topic}
            )
            
            discussion_messages.append(message)
            self.conversation_history.append(message)
            agent_info["message_count"] += 1
            agent_info["last_active"] = datetime.now()
            
            print(f"\n{agent.name} ({role.value}):")
            print(f"   {response_content[:150]}...")
            
        # Additional rounds: Responses and synthesis
        for round_num in range(2, max_rounds + 1):
            print(f"\nRound {round_num}: Building on Ideas")
            
            # Select a few agents to respond to others
            responding_agents = list(self.agents.items())[:2]  # First 2 agents respond
            
            for agent_id, agent_info in responding_agents:
                agent = agent_info["agent"]
                role = agent_info["role"]
                
                # Create synthesis prompt based on previous messages
                synthesis_prompt = self._create_synthesis_prompt(
                    topic, role, discussion_messages[-len(self.agents):]
                )
                
                if hasattr(agent, 'process_message'):
                    response_content = await agent.process_message(synthesis_prompt)
                    if hasattr(response_content, 'content'):
                        response_content = response_content.content
                else:
                    response_content = f"[{agent.name} synthesizes other perspectives as {role.value}]"
                
                message = ChatMessage(
                    sender=agent.name,
                    content=response_content,
                    role=role,
                    confidence=0.9,
                    metadata={"round": round_num, "topic": topic, "type": "synthesis"}
                )
                
                discussion_messages.append(message)
                self.conversation_history.append(message)
                
                print(f"\n{agent.name} (synthesis):")
                print(f"   {response_content[:150]}...")
        
        print(f"\nGroup discussion completed! Generated {len(discussion_messages)} messages")
        return discussion_messages
    
    def _create_role_prompt(self, topic: str, role: AgentRole, context: str) -> str:
        """Create a role-specific prompt for agents."""
        base_prompt = f"Topic for discussion: {topic}\n\n{context}\n\n"
        
        role_instructions = {
            AgentRole.FACILITATOR: "As a facilitator, guide the discussion and ask clarifying questions.",
            AgentRole.EXPERT: "As an expert, provide detailed knowledge and technical insights.",
            AgentRole.CRITIC: "As a critic, identify potential issues, limitations, and challenges.",
            AgentRole.SYNTHESIZER: "As a synthesizer, find connections and combine different viewpoints.",
            AgentRole.CREATIVE: "As a creative thinker, propose innovative and unconventional ideas."
        }
        
        return base_prompt + role_instructions.get(role, "Contribute your perspective on this topic.")
    
    def _create_synthesis_prompt(self, topic: str, role: AgentRole, previous_messages: List[ChatMessage]) -> str:
        """Create a prompt for synthesizing previous contributions."""
        context = f"Topic: {topic}\n\nPrevious contributions:\n"
        for msg in previous_messages:
            context += f"- {msg.sender}: {msg.content[:100]}...\n"
        
        return context + f"\n\nAs a {role.value}, build upon these ideas and add your synthesis:"
    
    def get_discussion_summary(self) -> Dict[str, Any]:
        """Get a comprehensive summary of the group discussion."""
        return {
            "conversation_id": self.conversation_id,
            "total_messages": len(self.conversation_history),
            "participants": {agent_id: info["message_count"] for agent_id, info in self.agents.items()},
            "topics_discussed": list(self.active_topics),
            "duration": f"{len(self.conversation_history)} message exchanges"
        }

# Create the group chat orchestrator
print("Initializing Group Chat Orchestrator...")
orchestrator = GroupChatOrchestrator()

# Add our existing generic agent with different roles (we'll create multiple instances)
orchestrator.add_agent(generic_agent, AgentRole.CREATIVE)

# Create a second instance of the generic agent for expert role
class ExpertAgent:
    """A wrapper around the generic agent to act as an expert."""
    def __init__(self, base_agent):
        self.name = "ExpertBot"
        self.base_agent = base_agent
    
    async def process_message(self, prompt: str) -> str:
        # Modify the prompt to emphasize expert behavior
        expert_prompt = f"As an expert providing technical analysis and insights: {prompt}"
        return await self.base_agent.process_message(expert_prompt)

expert_agent = ExpertAgent(generic_agent)
orchestrator.add_agent(expert_agent, AgentRole.EXPERT)

# Create additional specialized agents for the group
class SpecializedAgent:
    """A simple specialized agent for group chat demonstration."""
    def __init__(self, name: str, specialty: str):
        self.name = name
        self.specialty = specialty
    
    async def process_message(self, prompt: str) -> str:
        return f"[{self.name}, specialized in {self.specialty}, would provide expert insights on: {prompt[:50]}...]"

# Add specialized agents
critic_agent = SpecializedAgent("CriticBot", "identifying potential issues")
facilitator_agent = SpecializedAgent("FacilitatorBot", "guiding discussions")

orchestrator.add_agent(critic_agent, AgentRole.CRITIC)
orchestrator.add_agent(facilitator_agent, AgentRole.FACILITATOR)

print("Group Chat System ready with 4 agents!")

Initializing Group Chat Orchestrator...
Added Assistant as creative
Added ExpertBot as expert
Added CriticBot as critic
Added FacilitatorBot as facilitator
Group Chat System ready with 4 agents!


In [22]:
# Step 3.2: Run Group Chat Discussion
# Watch multiple agents collaborate on complex topics

async def run_group_chat_demo():
    """Run a demonstration of the group chat system."""
    
    discussion_topics = [
        "How can AI improve healthcare outcomes?",
        "What are the ethical considerations in autonomous vehicles?",
        "How should companies approach AI adoption?"
    ]
    
    print("Group Chat System Demonstration")
    print("Participants:")
    for agent_id, info in orchestrator.agents.items():
        print(f"   • {info['agent'].name} ({info['role'].value})")
    print()
    
    # Run discussion on the first topic
    selected_topic = discussion_topics[0]
    
    print(f"Discussion Topic: {selected_topic}")
    print("Starting collaborative discussion...")
    print()
    
    # Facilitate the discussion
    messages = await orchestrator.facilitate_discussion(
        topic=selected_topic,
        max_rounds=2  # Keep it manageable for demo
    )
    
    # Show summary
    print("\nDiscussion Summary:")
    summary = orchestrator.get_discussion_summary()
    for key, value in summary.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
    print("\nKey Benefits Demonstrated:")
    print("   ✓ Multiple perspectives on complex topics")
    print("   ✓ Role-based specialization")
    print("   ✓ Structured conversation flow")
    print("   ✓ Building upon others' ideas")
    
    return messages

print("Ready to run group chat demo!")
print("Run: await run_group_chat_demo()")

Ready to run group chat demo!
Run: await run_group_chat_demo()


In [23]:
# Step 3.3: Advanced Group Chat Features
# Explore sophisticated collaboration patterns

def analyze_group_dynamics():
    """Analyze the dynamics of the group chat discussion."""
    
    print("Group Chat Analysis")
    print("=" * 40)
    
    if not orchestrator.conversation_history:
        print("! No conversation history available. Run the group chat demo first.")
        return
    
    # Analyze participation patterns
    participation = {}
    role_distribution = {}
    
    for message in orchestrator.conversation_history:
        # Count messages per sender
        if message.sender not in participation:
            participation[message.sender] = 0
        participation[message.sender] += 1
        
        # Count messages per role
        role = message.role.value
        if role not in role_distribution:
            role_distribution[role] = 0
        role_distribution[role] += 1
    
    print("Participation Analysis:")
    for agent, count in participation.items():
        percentage = (count / len(orchestrator.conversation_history)) * 100
        print(f"   {agent}: {count} messages ({percentage:.1f}%)")
    
    print("\nRole Distribution:")
    for role, count in role_distribution.items():
        percentage = (count / len(orchestrator.conversation_history)) * 100
        print(f"   {role.title()}: {count} messages ({percentage:.1f}%)")
    
    print(f"\nConversation Metrics:")
    print(f"   Total Messages: {len(orchestrator.conversation_history)}")
    print(f"   Unique Participants: {len(participation)}")
    print(f"   Role Types: {len(role_distribution)}")
    print(f"   Active Topics: {len(orchestrator.active_topics)}")

def show_collaboration_patterns():
    """Demonstrate different collaboration patterns possible with the system."""
    
    patterns = {
        "Expert Consultation": "Bring in specialists for specific domain knowledge",
        "Devil's Advocate": "Use critics to challenge ideas and find weaknesses", 
        "Iterative Refinement": "Multiple rounds to polish and improve solutions",
        "Parallel Processing": "Different agents work on different aspects simultaneously",
        "Consensus Building": "Facilitators help find common ground between viewpoints",
        "Creative Brainstorming": "Creative agents generate innovative ideas",
        "Structured Analysis": "Systematic evaluation of complex problems"
    }
    
    print("Advanced Collaboration Patterns:")
    print("=" * 50)
    
    for pattern, description in patterns.items():
        print(f"{pattern}: {description}")
    
    print("\nNext Steps for Production:")
    production_features = [
        "User authentication and permissions",
        "Persistent conversation storage", 
        "Custom agent creation tools",
        "Advanced analytics and insights",
        "Real-time web interface",
        "Integration with external APIs",
        "Performance optimization for scale"
    ]
    
    for feature in production_features:
        print(f"   • {feature}")

print("Analysis tools ready!")
print("Run: analyze_group_dynamics()")
print("Run: show_collaboration_patterns()")

Analysis tools ready!
Run: analyze_group_dynamics()
Run: show_collaboration_patterns()


In [25]:
analyze_group_dynamics()
show_collaboration_patterns()

Group Chat Analysis
Participation Analysis:
   Assistant: 2 messages (33.3%)
   ExpertBot: 2 messages (33.3%)
   CriticBot: 1 messages (16.7%)
   FacilitatorBot: 1 messages (16.7%)

Role Distribution:
   Creative: 2 messages (33.3%)
   Expert: 2 messages (33.3%)
   Critic: 1 messages (16.7%)
   Facilitator: 1 messages (16.7%)

Conversation Metrics:
   Total Messages: 6
   Unique Participants: 4
   Role Types: 4
   Active Topics: 1
Advanced Collaboration Patterns:
Expert Consultation: Bring in specialists for specific domain knowledge
Devil's Advocate: Use critics to challenge ideas and find weaknesses
Iterative Refinement: Multiple rounds to polish and improve solutions
Parallel Processing: Different agents work on different aspects simultaneously
Consensus Building: Facilitators help find common ground between viewpoints
Creative Brainstorming: Creative agents generate innovative ideas
Structured Analysis: Systematic evaluation of complex problems

Next Steps for Production:
   • User

# Step 4: Design Your Own Agent
# Now it's your turn to create a unique agent!

def design_your_agent():
    """Guide users through designing their own agent."""
    
    print("Agent Design Workshop")
    print("=" * 30)
    
    print("Here are some agent ideas to inspire you:")
    
    agent_ideas = {
        "CodeReviewBot": {
            "description": "Reviews code for best practices, security issues, and optimization opportunities",
            "capabilities": ["technical_analysis", "security_review", "performance_optimization"],
            "roles": ["critic", "expert", "mentor"]
        },
        "CreativeWritingAssistant": {
            "description": "Helps writers with storytelling, character development, and creative techniques",
            "capabilities": ["creative_writing", "story_structure", "character_development"],
            "roles": ["creative", "coach", "collaborator"]
        },
        "DataAnalysisExpert": {
            "description": "Analyzes datasets, creates visualizations, and provides business insights",
            "capabilities": ["data_analysis", "visualization", "statistical_reasoning"],
            "roles": ["analyst", "advisor", "interpreter"]
        },
        "LearningTutor": {
            "description": "Personalized tutoring agent that adapts to individual learning styles",
            "capabilities": ["educational_content", "adaptive_learning", "progress_tracking"],
            "roles": ["teacher", "mentor", "evaluator"]
        },
        "StorytellerBot": {
            "description": "Interactive storytelling agent that creates immersive narrative experiences",
            "capabilities": ["narrative_creation", "character_interaction", "world_building"],
            "roles": ["narrator", "character", "world-builder"]
        }
    }
    
    print("Agent Ideas to Inspire You:")
    for name, details in agent_ideas.items():
        print(f"\n{name}: {details['description']}")
        print(f"   Capabilities: {', '.join(details['capabilities'])}")
        print(f"   Roles: {', '.join(details['roles'])}")
    
    print("\nYour Agent Design Framework:")
    framework = [
        "1. Define Purpose: What problem does your agent solve?",
        "2. Choose Capabilities: What types of responses does it need?",
        "3. Assign Roles: How will it behave in different contexts?",
        "4. Create Prompts: What instructions guide its behavior?",
        "5. Test & Iterate: How will you validate and improve it?"
    ]
    
    for step in framework:
        print(f"   {step}")
    
    print("\nImplementation Tips:")
    tips = [
        "Start simple and add complexity gradually",
        "Use the patterns from this workshop as templates", 
        "Test with diverse scenarios to find edge cases",
        "Consider combining generic and foundry capabilities",
        "Design for group collaboration if relevant"
    ]
    
    for tip in tips:
        print(f"   • {tip}")

design_your_agent()

In [ ]:
# 🎯 Final Challenge: Build Your Own Agent
# Apply what you've learned to create a custom agent

def design_your_agent():
    """Guide for designing a custom agent based on workshop learnings."""
    
    print("🎯 Design Your Own Agent Challenge!")
    print("=" * 50)
    
    agent_ideas = {
        "🏥 HealthBot": {
            "description": "Medical assistant with specialist routing",
            "capabilities": ["symptom analysis", "specialist referral", "health education"],
            "roles": ["diagnostician", "educator", "coordinator"]
        },
        "📚 StudyBuddy": {
            "description": "Educational agent with adaptive learning",
            "capabilities": ["concept explanation", "quiz generation", "progress tracking"],
            "roles": ["tutor", "motivator", "assessor"]
        },
        "💼 BusinessAnalyst": {
            "description": "Business intelligence with market analysis",
            "capabilities": ["data analysis", "trend prediction", "strategy recommendation"],
            "roles": ["analyst", "predictor", "advisor"]
        },
        "🎮 GameMaster": {
            "description": "Interactive storytelling with dynamic narratives",
            "capabilities": ["story generation", "character development", "choice consequences"],
            "roles": ["narrator", "character", "world-builder"]
        }
    }
    
    print("💡 Agent Ideas to Inspire You:")
    for name, details in agent_ideas.items():
        print(f"\n{name}: {details['description']}")
        print(f"   Capabilities: {', '.join(details['capabilities'])}")
        print(f"   Roles: {', '.join(details['roles'])}")
    
    print("\n🛠️ Your Agent Design Framework:")
    framework = [
        "1. 🎯 Define Purpose: What problem does your agent solve?",
        "2. 🧠 Choose Capabilities: What types of responses does it need?",
        "3. 🎭 Assign Roles: How will it behave in different contexts?",
        "4. 📝 Create Prompts: What instructions guide its behavior?",
        "5. 🧪 Test & Iterate: How will you validate and improve it?"
    ]
    
    for step in framework:
        print(f"   {step}")
    
    print("\n🚀 Implementation Tips:")
    tips = [
        "Start simple and add complexity gradually",
        "Use the patterns from this workshop as templates", 
        "Test with diverse scenarios to find edge cases",
        "Consider combining genetic evolution with foundry capabilities",
        "Design for group collaboration if relevant"
    ]
    
    for tip in tips:
        print(f"   • {tip}")

design_your_agent()

In [ ]:
# Next Steps: Advanced Learning Resources
# Continue your AI agent journey

def show_learning_path():
    """Display next steps for continued learning."""
    
    print("Continue Your AI Agent Journey")
    print("=" * 50)
    
    learning_tracks = {
        "Deep Learning Track": [
            "Study transformer architectures in detail",
            "Explore fine-tuning techniques for domain-specific agents", 
            "Learn about reinforcement learning for agent optimization",
            "Investigate multi-modal AI (text, vision, audio)"
        ],
        "Production Track": [
            "Master Azure AI services integration",
            "Learn containerization and deployment strategies",
            "Study load balancing and scaling patterns",
            "Implement monitoring and observability"
        ],
        "Collaboration Track": [
            "Advanced multi-agent coordination protocols",
            "Consensus mechanisms and conflict resolution",
            "Distributed agent architectures",
            "Human-AI collaboration patterns"
        ],
        "Specialization Track": [
            "Domain-specific agent development",
            "Custom training data preparation",
            "Evaluation metrics and benchmarking",
            "Ethical AI and bias mitigation"
        ]
    }
    
    for track, topics in learning_tracks.items():
        print(f"\n{track}:")
        for topic in topics:
            print(f"   • {topic}")
    
    print("\nRecommended Resources:")
    resources = [
        "LangChain Documentation: Comprehensive guides and examples",
        "Azure AI Documentation: Enterprise AI implementation",
        "AI Research Papers: Latest developments in agent systems",
        "Open Source Projects: Real-world agent implementations",
        "Academic Courses: Formal education in AI/ML",
        "Developer Communities: Connect with other AI practitioners"
    ]
    
    for resource in resources:
        print(f"   • {resource}")

show_learning_path()

print("\nThank you for completing the LangChain Agents Workshop!")
print("You're now equipped to build amazing AI systems!")
print("Go forth and create intelligent agents that make the world better!")

## Section 6: Create Azure AI Foundry Agent

**The Grand Finale!** Let's create a production-ready agent using Azure AI Foundry. This agent will have:

- **Enterprise security**: Managed identity and secure connections
- **Advanced monitoring**: Built-in analytics and logging  
- **Production features**: Scalability and reliability
- **Rich capabilities**: Advanced reasoning and tool usage

### Exercise 4: Build Your Azure AI Foundry Agent

In [ ]:
# Create Azure AI Foundry agent configuration
foundry_agent_config = AgentConfig(
    name="workshop_foundry_agent",
    agent_type="azure_foundry",
    enabled=True,
    instructions="""You are an advanced AI agent powered by Azure AI Foundry, designed for enterprise-grade applications.

ENTERPRISE CAPABILITIES:
- Advanced reasoning and problem-solving
- Integration with Azure ecosystem
- Built-in security and compliance
- Production-ready scalability
- Comprehensive monitoring and analytics

WORKSHOP ROLE:
- Demonstrate enterprise AI capabilities
- Explain Azure AI Foundry benefits
- Provide production-ready examples
- Show integration possibilities

RESPONSE STYLE:
- Professional yet approachable
- Include technical details when relevant
- Highlight enterprise features
- Provide actionable insights
- Use examples from real-world scenarios""",
    metadata={
        "description": "Production-ready Azure AI Foundry agent",
        "capabilities": [
            "enterprise_reasoning",
            "azure_integration", 
            "security_compliance",
            "production_monitoring",
            "advanced_analytics",
            "scalable_deployment"
        ],
        "workshop_level": "advanced",
        "environment": "azure_foundry"
    },
    framework_config={
        "provider": "azure_foundry",
        "model": "gpt-4o",
        "temperature": 0.6,  # Balanced for enterprise use
        "max_tokens": 1200,  # Detailed enterprise responses
        "endpoint": os.getenv("PROJECT_ENDPOINT"),
        "use_managed_identity": True,  # Enterprise security
        "enable_monitoring": True,     # Production monitoring
        "enable_analytics": True       # Usage analytics
    }
)

print("🏢 Azure AI Foundry Agent Configuration Created!")
print("🔑 Key enterprise features:")
print(f"  ✅ Managed Identity: {foundry_agent_config.framework_config.get('use_managed_identity')}")
print(f"  ✅ Monitoring: {foundry_agent_config.framework_config.get('enable_monitoring')}")
print(f"  ✅ Analytics: {foundry_agent_config.framework_config.get('enable_analytics')}")
print(f"  ✅ Provider: {foundry_agent_config.framework_config.get('provider')}")
print(f"  ✅ Endpoint: {foundry_agent_config.framework_config.get('endpoint')}")

In [ ]:
# Create and initialize the Azure AI Foundry agent
try:
    # Use our LangChain Azure Foundry agent implementation
    foundry_agent = LangChainAzureFoundryAgent(foundry_agent_config)
    await foundry_agent.initialize()
    
    print("🚀 Azure AI Foundry Agent Created Successfully!")
    
    # Register in our agent registry
    agent_registry.register_agent("foundry", foundry_agent)
    
    print(f"📋 Agent Registry now contains: {agent_registry.get_all_agents()}")
    print(f"🎯 Foundry agent capabilities: {foundry_agent.get_capabilities()}")
    
    # Show enterprise features
    print("\n🏢 Enterprise Features Enabled:")
    print("  ✅ Secure authentication with managed identity")
    print("  ✅ Built-in request/response monitoring")
    print("  ✅ Automatic retry logic with exponential backoff")
    print("  ✅ Integration with Azure security services")
    print("  ✅ Compliance and governance features")
    print("  ✅ Production-ready scalability")
    
except Exception as e:
    print(f"❌ Error creating Azure AI Foundry agent: {e}")
    print("This might happen if Azure AI Foundry isn't fully configured")
    print("But we can still demonstrate the configuration approach!")

In [ ]:
# Test the Azure AI Foundry agent
print("🧪 Testing Azure AI Foundry Agent...")

if 'foundry_agent' in locals():
    # Test enterprise features
    enterprise_tests = [
        "What are the key benefits of using Azure AI Foundry for enterprise AI applications?",
        "How does managed identity improve security in AI applications?",
        "Can you explain the monitoring and analytics capabilities you provide?",
        "What makes you different from the basic agents we created earlier?"
    ]
    
    for i, test_message in enumerate(enterprise_tests, 1):
        print(f"\n🔬 Test {i}/{len(enterprise_tests)}")
        try:
            response = await foundry_agent.process_message(test_message, [], {
                "test_id": f"enterprise_test_{i}",
                "workshop_session": "langchain_foundry"
            })
            
            print(f"💬 Question: {test_message}")
            print(f"🏢 Foundry Agent: {response.content}")
            print(f"📊 Enterprise Metadata: {response.metadata}")
            print("-" * 80)
            
        except Exception as e:
            print(f"❌ Test {i} failed: {e}")
            
else:
    print("⚠️ Foundry agent not available for testing")
    print("In a real environment, this would demonstrate:")
    print("  - Advanced reasoning capabilities")
    print("  - Enterprise security features")
    print("  - Built-in monitoring and analytics")
    print("  - Production-ready performance")

## Section 7: Compare Agent Performances

🏆 **Time for the Grand Comparison!** Let's compare all the agents we've built and see how they perform on the same tasks.

This section will help you understand:
- The evolution from basic to enterprise agents
- Performance differences between implementations  
- When to use each type of agent
- Real-world application scenarios

In [ ]:
# Comprehensive agent comparison
import time
from typing import Dict, List, Tuple

async def compare_agents(test_message: str, agents_dict: Dict[str, IAgent]) -> Dict[str, Dict]:
    """Compare multiple agents on the same task."""
    results = {}
    
    print(f"🔬 Testing all agents with: '{test_message}'")
    print("=" * 80)
    
    for agent_name, agent in agents_dict.items():
        try:
            start_time = time.time()
            
            # Test the agent
            response = await agent.process_message(test_message, [], {
                "comparison_test": True,
                "agent_name": agent_name
            })
            
            end_time = time.time()
            response_time = round((end_time - start_time) * 1000, 2)  # Convert to milliseconds
            
            # Store results
            results[agent_name] = {
                "response": response.content,
                "response_time_ms": response_time,
                "capabilities": agent.get_capabilities(),
                "metadata": response.metadata,
                "success": True
            }
            
            # Display results
            print(f"🤖 {agent_name.upper()} AGENT:")
            print(f"   Response Time: {response_time}ms")
            print(f"   Response: {response.content[:150]}{'...' if len(response.content) > 150 else ''}")
            print(f"   Capabilities: {agent.get_capabilities()}")
            print("-" * 60)
            
        except Exception as e:
            results[agent_name] = {
                "error": str(e),
                "success": False,
                "response_time_ms": 0
            }
            print(f"❌ {agent_name.upper()} AGENT: Error - {e}")
            print("-" * 60)
    
    return results

# Prepare agents for comparison
agents_to_compare = {}

# Add available agents
if 'basic_agent' in locals():
    agents_to_compare["basic"] = basic_agent
    
if 'enhanced_agent' in locals():
    agents_to_compare["enhanced"] = enhanced_agent
    
if 'foundry_agent' in locals():
    agents_to_compare["foundry"] = foundry_agent

print(f"🎯 Comparing {len(agents_to_compare)} agents:")
for name in agents_to_compare.keys():
    print(f"   ✅ {name.title()} Agent")

In [ ]:
# Test 1: Basic conversation
print("🧪 TEST 1: Basic Conversation")
results_1 = await compare_agents(
    "Hello! Can you explain what makes a good AI agent?", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 2: Technical explanation
print("🧪 TEST 2: Technical Explanation")
results_2 = await compare_agents(
    "Explain the benefits of using dependency injection in software architecture.", 
    agents_to_compare
)

print("\n" + "="*80 + "\n")

# Test 3: Enterprise scenario
print("🧪 TEST 3: Enterprise Scenario")
results_3 = await compare_agents(
    "How would you design a scalable AI system for a large enterprise with security and compliance requirements?", 
    agents_to_compare
)

In [ ]:
# Performance analysis
print("📊 PERFORMANCE ANALYSIS")
print("="*80)

def analyze_results(test_name: str, results: Dict):
    """Analyze and display test results."""
    print(f"\n📈 {test_name} Analysis:")
    
    successful_agents = {k: v for k, v in results.items() if v.get('success', False)}
    
    if successful_agents:
        # Response time analysis
        avg_response_time = sum(v['response_time_ms'] for v in successful_agents.values()) / len(successful_agents)
        fastest_agent = min(successful_agents.items(), key=lambda x: x[1]['response_time_ms'])
        
        print(f"   ⚡ Average response time: {avg_response_time:.2f}ms")
        print(f"   🏃 Fastest agent: {fastest_agent[0]} ({fastest_agent[1]['response_time_ms']}ms)")
        
        # Capability analysis
        all_capabilities = set()
        for agent_data in successful_agents.values():
            all_capabilities.update(agent_data.get('capabilities', []))
        
        print(f"   🎯 Total unique capabilities: {len(all_capabilities)}")
        print(f"   📋 Capabilities: {', '.join(sorted(all_capabilities))}")
        
        # Response quality (length as a proxy)
        response_lengths = {k: len(v['response']) for k, v in successful_agents.items()}
        most_detailed = max(response_lengths.items(), key=lambda x: x[1])
        
        print(f"   📝 Most detailed response: {most_detailed[0]} ({most_detailed[1]} characters)")
    
    else:
        print("   ❌ No successful responses for this test")

# Analyze all tests
if 'results_1' in locals():
    analyze_results("Basic Conversation", results_1)
    
if 'results_2' in locals():
    analyze_results("Technical Explanation", results_2)
    
if 'results_3' in locals():
    analyze_results("Enterprise Scenario", results_3)

## 🎉 Congratulations! Workshop Complete!

You've successfully completed the LangChain Agents Workshop! Here's what you've accomplished:

### ✅ **What You've Built:**
1. **Basic Generic Agent** - Simple conversational AI
2. **Enhanced Agent** - With memory and advanced capabilities  
3. **Azure AI Foundry Agent** - Enterprise-ready with security and monitoring

### 🎯 **Key Learnings:**
- **Modern Architecture**: Plugin-based, extensible design
- **Configuration-Driven**: Easy to modify and deploy
- **Security Best Practices**: Using managed identity and secure connections
- **Enterprise Features**: Monitoring, analytics, and scalability

### 🚀 **Next Steps:**
1. **Experiment**: Try different configurations and instructions
2. **Extend**: Add custom tools and capabilities to your agents
3. **Deploy**: Use Azure AI Foundry for production deployment
4. **Monitor**: Implement logging and analytics for your agents

### 📚 **Resources:**
- [Azure AI Foundry Documentation](https://docs.microsoft.com/azure/ai-foundry/)
- [LangChain Documentation](https://python.langchain.com/)
- [Modern Agent Architecture Guide](../../README.md)
- [Configuration Examples](../../examples/)

### 🤝 **Questions & Discussion:**
What questions do you have about building and deploying AI agents?

**Thank you for participating in this workshop!** 🎊

## Section 7: LangChain vs Semantic Kernel - Framework Comparison

Now that you've experienced both workshops, let's compare the frameworks to help you choose the right one for your projects.

In [ ]:
def create_framework_comparison():
    """
    Create a comprehensive comparison between LangChain and Semantic Kernel.
    """
    
    print("🆚 LANGCHAIN vs SEMANTIC KERNEL COMPARISON")
    print("=" * 50)
    
    # Framework comparison matrix
    comparison_data = {
        "Aspect": [
            "Architecture", "Learning Curve", "Tool Ecosystem", "Memory Management",
            "Multi-Provider Support", "Enterprise Features", "Community Size",
            "Microsoft Integration", "Flexibility", "Performance", 
            "Documentation", "Production Readiness"
        ],
        "LangChain": [
            "Chain-based", "Moderate", "Extensive", "Advanced",
            "Excellent", "Good", "Large",
            "Good", "Very High", "Good",
            "Excellent", "Mature"
        ],
        "Semantic Kernel": [
            "Plugin-based", "Easy", "Growing", "Basic",
            "Good", "Excellent", "Medium",
            "Native", "High", "Optimized",
            "Good", "Enterprise-Ready"
        ]
    }
    
    print("\\n📊 DETAILED COMPARISON")
    print("-" * 25)
    
    # Print comparison table
    col_widths = [20, 15, 18]
    headers = ["Aspect", "LangChain", "Semantic Kernel"]
    
    # Print header
    header_row = ""
    for i, header in enumerate(headers):
        header_row += f"{header:<{col_widths[i]}}"
    print(header_row)
    print("-" * sum(col_widths))
    
    # Print rows
    for i, aspect in enumerate(comparison_data["Aspect"]):
        row = f"{aspect:<{col_widths[0]}}"
        row += f"{comparison_data['LangChain'][i]:<{col_widths[1]}}"
        row += f"{comparison_data['Semantic Kernel'][i]:<{col_widths[2]}}"
        print(row)
    
    print("\\n🎯 WHEN TO CHOOSE LANGCHAIN")
    print("-" * 30)
    
    langchain_use_cases = [
        "🔗 Complex chain orchestration and workflows",
        "🛠️ Need extensive pre-built tool integrations",
        "🧠 Advanced memory and retrieval requirements",
        "🌐 Multi-provider flexibility is critical",
        "📚 Rich documentation and community support needed",
        "🔄 Rapid prototyping with diverse components",
        "🐍 Python-first development approach"
    ]
    
    for use_case in langchain_use_cases:
        print(f"   {use_case}")
    
    print("\\n🎯 WHEN TO CHOOSE SEMANTIC KERNEL")
    print("-" * 35)
    
    sk_use_cases = [
        "🏢 Enterprise Microsoft environment",
        "🚀 Quick start with minimal learning curve",
        "🔌 Plugin-based extensibility preferred",
        "⚡ Performance optimization important",
        "🛡️ Enterprise security and compliance focus",
        "🔗 Native Azure integration required",
        "🎯 Simpler, more focused agent requirements"
    ]
    
    for use_case in sk_use_cases:
        print(f"   {use_case}")
    
    print("\\n🤝 HYBRID APPROACH")
    print("-" * 17)
    print("🔄 You can use both frameworks in the same project!")
    print("   • LangChain for complex workflows and tools")
    print("   • Semantic Kernel for Microsoft-integrated components")
    print("   • Choose based on specific use case requirements")
    
    print("\\n🎓 LEARNING RECOMMENDATION")
    print("-" * 25)
    print("📚 Start with: Semantic Kernel (easier learning curve)")
    print("🔄 Then explore: LangChain (for advanced capabilities)")
    print("🎯 Choose based on: Your specific project needs")
    print("💡 Remember: Both are excellent frameworks!")
    
    return comparison_data

# Generate the comparison
comparison_results = create_framework_comparison()

print("\\n✨ Framework comparison complete!")
print("🎯 Now you can make informed decisions about which framework to use!")
print("🚀 Both workshops completed - you're ready for production AI agents!")

## Optional utilities

Use these helpers if you need to:
- Reset or clean up your environment variables and optional .env file
- Start the LangChain FastAPI server (uvicorn) from the notebook
- Stop the server safely on Windows

These are optional and independent from the Quick Start steps above. If you don't need them, you can ignore this section.

In [ ]:
# Utility: Reset config (.env and in-memory)
import os, json, shutil
from pathlib import Path

# Toggle deletion of .env file created in Step 2
DELETE_ENV_FILE = False  # set True to remove .env

# Env vars used by this LangChain app
_ENV_KEYS = [
    "AZURE_INFERENCE_ENDPOINT","AZURE_INFERENCE_CREDENTIAL","GENERIC_MODEL",
    "PROJECT_ENDPOINT","PEOPLE_AGENT_ID","KNOWLEDGE_AGENT_ID",
    "FRONTEND_URL","LOG_LEVEL","ENVIRONMENT",
    "SESSION_STORAGE_TYPE","SESSION_STORAGE_PATH","REDIS_URL",
    "DEBUG_LOGS","CONFIG_PATH"
 ]

def _mask(v):
    if v is None:
        return ""
    return v[:4] + "***" if len(v) > 8 else "***"

def reset_config(delete_env: bool = False):
    # Clear in-memory env
    cleared = {}
    for k in _ENV_KEYS:
        if k in os.environ:
            cleared[k] = os.environ.pop(k)
    # Optionally delete .env in this folder
    env_path = Path(".env")
    removed_env_file = False
    if delete_env and env_path.exists():
        try:
            env_path.unlink()
            removed_env_file = True
        except Exception as e:
            print(f"Warning: couldn't delete .env: {e}")
    print("Cleared env keys:")
    for k, v in cleared.items():
        print(f"- {k} = { _mask(v) }")
    print(f"Removed .env file: {removed_env_file}")
    return {"cleared": list(cleared.keys()), "removed_env_file": removed_env_file}

result = reset_config(DELETE_ENV_FILE)
print("Reset complete.")

In [ ]:
# Utility: Start API server (uvicorn) in background
import os, sys, subprocess, time, json
from pathlib import Path

# Settings
HOST = os.getenv("HOST", "127.0.0.1")
PORT = int(os.getenv("PORT", "8001"))  # avoid conflict with SK if it's 8000
RELOAD = False  # set True during local dev
LOG_LEVEL = os.getenv("LOG_LEVEL", "info")
PID_FILE = Path(".uvicorn_pid")

def start_server():
    if PID_FILE.exists():
        print("A server appears to be running already (PID file exists). If it's stale, run the Stop cell first.")
        return {"status": "skipped", "reason": "pid_exists"}
    cmd = [sys.executable, "-m", "uvicorn", "main:app", "--host", HOST, "--port", str(PORT), "--log-level", LOG_LEVEL]
    if RELOAD:
        cmd.append("--reload")
    # On Windows, creationflags=CREATE_NEW_PROCESS_GROUP helps Ctrl+C and termination
    creationflags = 0x00000200  # CREATE_NEW_PROCESS_GROUP
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, creationflags=creationflags)
    PID_FILE.write_text(str(proc.pid))
    print(f"Starting uvicorn main:app at http://{HOST}:{PORT} (pid={proc.pid})...")
    # Brief wait to give server time to bind
    time.sleep(1.5)
    return {"status": "started", "pid": proc.pid, "url": f"http://{HOST}:{PORT}"}

result = start_server()
result

In [ ]:
# Utility: Stop API server (read PID file and terminate)
import os, signal
from pathlib import Path

PID_FILE = Path(".uvicorn_pid")

def stop_server():
    if not PID_FILE.exists():
        print("No PID file found. If a server is running, you may need to stop it manually.")
        return {"status": "skipped", "reason": "no_pid"}
    try:
        pid = int(PID_FILE.read_text().strip())
    except Exception as e:
        print(f"Couldn't read pid: {e}")
        return {"status": "error", "error": str(e)}
    try:
        # Windows-friendly termination: first try CTRL_BREAK_EVENT, then terminate
        try:
            os.kill(pid, signal.CTRL_BREAK_EVENT)
        except Exception:
            # Fallback to terminate
            os.kill(pid, signal.SIGTERM)
        print(f"Sent termination to pid {pid}")
    except Exception as e:
        print(f"Error signaling process: {e}")
        return {"status": "error", "error": str(e)}
    try:
        PID_FILE.unlink()
    except Exception:
        pass
    return {"status": "stopped", "pid": pid}

result = stop_server()
result